# Introduction to Reinforcement Learning
---

## Setup

<img src="fig/rl.png" width="300" alt="RL Setup"/>


Markov Decision Process $M := (\mathcal{S}, \mathcal{A}, P, r)$ 

$\mathcal{S}$ is the state space, $\mathcal{A}$ is the finite action space

$P$ is the transition probability kernel

$r(s,a)$ is the reward function mapping state-action pairs to real-valued subset

Stochastic policy $\pi$ as probability distribution over actions given a state $\pi(a|s)$

### Goal

Maximise w.r.t $\pi$ the expected discounted total reward:
    
$R = \sum \limits_{t=0}^{\infty} \gamma^t r_t$

$\gamma$ is the discount factor that trades-off the importance of immediate and future rewards, $0<\gamma<1$

## Reinforcement Learning for a Markov Decision Process

*Action-value function* $Q^{\pi}$ of a policy $\pi$: 
$Q^{\pi}(s_t, a_t) = \mathbb{E}_{a \sim \pi, s \sim P} [\sum_{l=0}^{\infty} \gamma^l r_{t+l} | s_t, a_t]$

The $Q$ values under policy $\pi$ represent the total discounted return from taking a given action at a given state and following the policy onwards.

*State-value function* $V^{\pi}$ of a policy $\pi$: 
$V^{\pi}(s_t) = \mathbb{E}_{a \sim \pi} Q^{\pi}(s_t, a)$

The $V$ value of a state under policy $\pi$ is the expected total discounted return obtained by following $\pi$ from that state.

### Q-learning

Given $Q$ function, the agent knows which action is best in a given state

Greedy policy is the policy that takes action with the highest $Q$, $\pi^* = argmax_a Q(s,a)$

### Bellman equation

$Q$ value and $V$ value can be decomposed using *Bellman equation*:

$Q^{\pi}(s,a) = \mathbb{E}_{\pi} [r_t + \gamma Q^{\pi}(s_{t+1},a_{t+1}) | s_t=s, a_t=a]$

$V^{\pi}(s) = \mathbb{E}_{\pi} [r_t + \gamma V^{\pi}(s_{t+1}) | s_t=s]$    

Note: after time $t$, actions are chosen by $\pi$ so in expectation we could replace $Q^{\pi}(s_{t+1}, a_{t+1})$ by $V^{\pi}(s_{t+1})$

## Bellman Optimality Equation
    
The optimal policy $\pi^*$ must satisfy

$Q^{\pi^*}(s,a) = \mathbb{E}_{\pi^*} [r_t + \gamma \max_a Q^{\pi^*}(s_{t+1},a)]$

**Algorithm**: 
While the equality does not hold, alternate between two steps:    
    
1. Policy Evaluation: estimate $Q$ function for a fixed $\pi$
2. Policy Improvement: improve $\pi$ by acting greedily on $Q$
   
<img src="fig/evalimprov.png" width="480" alt="Policy Evaluation - Policy Improvement"/>


Bootstraping: use your $Q$ values estimation to compute *target*, i.e. right-hand side of Bellman equation

Guaranteed to converge in tabular case (contraction argument), but not in approximation case

source: [R. S. Sutton, Reinforcement Learning: An introduction](http://incompleteideas.net/book/bookdraft2017nov5.pdf)

## Deep Q Learning (DQN)

Approximate $Q$ function using a neural network:

$Q^{\pi^*}(s,a) = \mathbb{E}_{\pi^*} [r_t + \gamma \max_a Q^{\pi^*}(s_{t+1},a)]$

* $Q$-value is estimated using a neural network
* $Q$ network is trained over mini-batches of state-action pairs
* Expectation can be approximated using Monte Carlo sampling (in practice, just one sample)

Reference: [Human-level control through deep reinforcement learning](https://www.nature.com/articles/nature14236)

## Experience Replay

The training dataset of state-action pairs is crucial for the performance

* Collect data using a fixed policy with an *exploration mechanism*
* Collected samples are highly correlated. To mitigate this, experience replay is an idea to keep a common memory over several episodes and randomly sample batches from there
* In prioritized experienced replay, sampling is weighted to replay important transitions more frequently

Reference: [Prioritized Experience Replay](https://arxiv.org/abs/1511.05952)

## Double Deep Q Learning (DDQN)

**Problem**: overestimation bias of Q-values 
$\mathbb{E}_{\epsilon}[\max_a (Q(s,a) + \epsilon)] \geq \max_a Q(s,a)$ 

**Idea**: use two independent estimators $Q_1$ and $Q_2$ to make unbiased value estimates

**In practice**: In Double DQN *target network* is used to estimate the right-hand side of Bellman update. Policy is greedy one based on *current network* rather than the target network. Target network is synced with current network every several episodes to simulate "independent" estimation.

$Q_{1}^{\pi^*}(s,a) = \mathbb{E}_{\pi^*} [r_t + \gamma \max_a Q_2^{\pi^*}(s_{t+1},a)]$

Reference: [Deep Reinforcement Learning with Double Q-learning](https://papers.nips.cc/paper/3964-double-q-learning)

## Dueling network

**Problem**: overestimation bias of Q-values 

**Idea**: reduce variance of Q-value estimates by subtracting a state value (constant for all actions)

* $A(a,s) = Q(s,a) - V(s)$
* $A(a,s)$ is called *advantage function*

**In practice**: design a neural network that has two flows -- one for $Q$ value estimation and the other one for $V$ value.

<img src="fig/dueling_network.png" width="480" alt="Dueling Network"/>

Reference: [Dueling Network Architectures for Deep Reinforcement Learning](http://proceedings.mlr.press/v48/wangf16.pdf)

## Actor Critic

**Problem**: argmax of a greedy policy is not differentiable: small error in $Q$ estimation might result in argmax being arbitrarily far

**Idea**: parametrize policy $\pi = \pi_{\theta}(a|s)$, policy improvement step becomes optimization step over $\theta$

**In practice**: two networks -- policy network and value network ($Q$ or $A$ value)

* $a \sim \pi_{\theta}(a|s)$
* $Q^{\pi_{\theta}}(s,a) = \mathbb{E}_{\pi_{\theta}} [r_t + \gamma \max_a Q^{\pi_{\theta}}(s_{t+1},a)]$
* $\theta \leftarrow \theta + \alpha \nabla_{\theta} \log \pi_{\theta}(a|s)Q^{\pi_{\theta}}(s,a)$

**Connection to Policy Gradient**: reduce variance of policy gradient by replacing the true reward by its estimated mean, therefore introducing bias

## Resources

[R. S. Sutton, Reinforcement Learning: An introduction](http://incompleteideas.net/book/bookdraft2017nov5.pdf)

[Introduction into RL by R. Munos](http://videolectures.net/netadis2013_munos_reinforcement_learning/)

[ICML 2017 Deep RL tutorial by S. Levine and C. Finn](https://sites.google.com/view/icml17deeprl)